In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.fft import rfft, rfftfreq, irfft

In [2]:
def load_gsignal(fp):
    s = np.load(fp)
    return s

def draw_gsignal(arrays, time=None, gtype='scatter'):
    fig = plt.figure(figsize=(18, 8), dpi=200)
    fig.patch.set_facecolor('white')

    for i, array in enumerate(arrays):
        if time is None:
            time = list(range(len(array)))
        if gtype == 'scatter':
            plt.scatter(time, array, label=i)
        elif gtype == 'line':
            plt.plot(time, array, label=i)
    plt.xlabel('')
    plt.ylabel('')
    plt.legend()
    plt.show()
    plt.clf()

In [3]:
s1 = load_gsignal("././00000e74ad.npy")
draw_gsignal(s1)

In [4]:
N = s1.shape[1]
# sample spacing
sample_rate = 2048.0
T = 1.0 / sample_rate

signal = s1[0]
yf = rfft(signal)
xf = rfftfreq(N, T)
# xf = fftshift(xf)
# yplot = fftshift(yf)

draw_gsignal(time=xf, arrays=[np.abs(yf)], gtype='line')

In [5]:
# The maximum frequency is half the sample rate
points_per_freq = len(xf) / (sample_rate / 2)
target_idx = int(points_per_freq * 35)
target_idx_right = int(points_per_freq * 150)
yf[target_idx:target_idx_right] = 0

plt.plot(xf, np.abs(yf))
plt.show()

In [6]:
new_sig = irfft(yf)

plt.plot(new_sig)
plt.show()

In [7]:
eps = signal - new_sig
plt.plot(eps)
plt.show()

In [8]:
from scipy.signal import butter, sosfilt, sosfreqz

def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        sos = butter(order, [low, high], analog=False, btype='band', output='sos')
        return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        sos = butter_bandpass(lowcut, highcut, fs, order=order)
        y = sosfilt(sos, data)
        return y

new_sig = butter_bandpass_filter(signal, 35, 600, sample_rate, order=2)

plt.plot(new_sig)
plt.plot(s1[2])
plt.show()


In [9]:
import filters

signal = load_gsignal("././00000e74ad.npy")
sample_rate = 2048.0
bp = filters.bandpass(35, 600, sample_rate)
notches = [filters.notch(line, sample_rate) for line in (60, 120, 180)]
zpk = filters.concatenate_zpks(bp, *notches)
hfilt = filters.apply_filter(signal[1], *zpk)
draw_gsignal([hfilt], gtype='line')

In [219]:
import h5py
import filters

signal = None
with h5py.File('./H-H1_LOSC_4_V1-1126256640-4096.hdf5', 'r') as f:
    d = f['strain']['Strain']
    signal = d[0:]

sample_rate = 4096
bp = filters.bandpass(50, 250, sample_rate)
notches = [filters.notch(line, sample_rate) for line in (60, 120, 180)]
zpk = filters.concatenate_zpks(bp, *notches)
hfilt = filters.apply_filter(signal, *zpk)
print(len(hfilt)/sample_rate)
#hfilt = hfilt[13*sample_rate:1800*sample_rate]
draw_gsignal([hfilt[2822*sample_rate:2822*sample_rate+int(0.6*sample_rate)]], gtype='line')

In [223]:
s = signal[2822*sample_rate:2826*sample_rate]
f_asd, v_asd = filters.asd(s, sample_rate, 4, 2)

fig, ax = plt.subplots()
ax.semilogy(f_asd, v_asd)
ax.set_xscale('symlog')
plt.xlabel('frequency [Hz]')
plt.ylabel('ASD [Hz^{-1/2}]')
plt.show()

In [12]:
s = signal[2822*sample_rate:2826*sample_rate]

fftlength = 2
overlap = 0.5 * fftlength
f_asd, v_asd = filters.asd(s, sample_rate, fftlength=fftlength, overlap=overlap)

fmin = 10
fmax = 2000
fig, ax = plt.subplots()
ax.semilogy(f_asd, v_asd)
ax.set_xscale('symlog')
plt.axis([fmin, fmax, 1e-24, 1e-19])
plt.xlabel('frequency [Hz]')
plt.ylabel('ASD [Hz^{-1/2}]')
plt.show()

whiten_s = filters.whiten(s, sample_rate, fftlength=fftlength, overlap=overlap)

fig, ax = plt.subplots()
ax.plot(whiten_s)
plt.xlabel('frequency [Hz]')
plt.ylabel('ASD [Hz^{-1/2}]')
plt.show()

f_asd, v_asd = filters.asd(whiten_s, sample_rate, fftlength=fftlength, overlap=overlap)

fig, ax = plt.subplots()
ax.semilogy(f_asd, v_asd)
ax.set_xscale('symlog')
plt.xlabel('frequency [Hz]')
plt.ylabel('ASD [Hz^{-1/2}]')
plt.show()

zpk = filters.butter_bandpass(35, 350, sample_rate, 8)
whiten_sf = filters.apply_filter(whiten_s, *zpk)

fig, ax = plt.subplots()
ax.plot(whiten_sf[0:int(0.6*sample_rate)])
plt.xlabel('frequency [Hz]')
plt.ylabel('ASD [Hz^{-1/2}]')
plt.show()

In [13]:
import matplotlib.mlab as mlab
from scipy.interpolate import interp1d

NFFT = 2*sample_rate
fmin = 10
fmax = 2000
Pxx_H1, freqs = mlab.psd(s, Fs = sample_rate, NFFT = NFFT)

# We will use interpolations of the ASDs computed above for whitening:
psd_H1 = interp1d(freqs, Pxx_H1)

# plot the ASDs:
plt.figure()
plt.loglog(freqs, np.sqrt(Pxx_H1),'r',label='H1 strain')
plt.axis([fmin, fmax, 1e-24, 1e-19])
plt.grid('on')
plt.ylabel('ASD (strain/rtHz)')
plt.xlabel('Freq (Hz)')
plt.legend(loc='upper center')
plt.title('Advanced LIGO strain data near GW150914')
plt.show()

In [14]:
def whiten(strain, interp_psd, dt):
    Nt = len(strain)
    freqs = np.fft.rfftfreq(Nt, dt)

    # whitening: transform to freq domain, divide by asd, then transform back,
    # taking care to get normalization right.
    hf = np.fft.rfft(strain)
    white_hf = hf / (np.sqrt(interp_psd(freqs) /dt/2.))
    white_ht = np.fft.irfft(white_hf, n=Nt)
    return white_ht

dt = 1./sample_rate
# now whiten the data from H1 and L1, and also the NR template:
strain_H1_whiten = whiten(s,psd_H1,dt)

In [15]:
from scipy.signal import butter, sosfiltfilt

sos = butter(4, [50.*2./sample_rate, 250.*2./sample_rate], btype='band', output='sos')
strain_H1_whitenbp = sosfiltfilt(sos, strain_H1_whiten)

plt.figure()
plt.plot(strain_H1_whitenbp[int(1.8*sample_rate): int(2.3*sample_rate)],'r',label='H1 strain')
plt.ylabel('whitented strain')
plt.legend(loc='lower left')
plt.title('Advanced LIGO WHITENED strain data near GW150914')
plt.show()